# Projeto 1 - Ciência dos Dados

Nome: Layne Silva

Nome: Lidia Domingos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [104]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from string import punctuation
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lidia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [105]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lidia\Desktop\Ciência dos Dados\Projeto1\PROJETO-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [106]:
filename = 'XIAOMI.xlsx'

In [107]:
train = pd.read_excel(filename)
train.head(5)

,TREINAMENTO,CLASSIFICAÇÃO
0,quem aqui tb pronuncia xiaomi xaiomi,0
1,xiaomi mi true airdots 2s earphone(aliexpress)...,0
2,@felpsflip carreguei meu xiaomi uma vez na vid...,1
3,"#promo #\n$133.99 ~ 111,85 € ✂️cupão/cupom/cup...",0
4,"#promo #\n$299.99 ~ 250,41 € ✂️cupão/cupom/cup...",0


In [108]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0
3,comprei mais capinha pro xiaomi https://t.co/7...,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

*Escolhida a empresa XIAOMI, classificamos como tweets relevantes toda e qualquer citação (seja ela positiva ou negativa) dos internautas falando sobre o produto. Tweets considerados como irrelevantes seriam tweets não relacionados diretamente ao produto.*

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [109]:
#Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.
#Correção de espaços entre palavras e/ou emojis.
#Proposta de outras limpezas/transformações que não afetem a qualidade da informação.

import re 
from nltk.tokenize.casual import TweetTokenizer #separa os emojis como palavras.

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-@.[\]:"?;\),("/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '',text_subbed)
    t = TweetTokenizer()
    text_subbed = t.tokenize(text_subbed) 
    return text_subbed

def RemoveStopWords(list_of_words):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    filtered = []
    for word in list_of_words:
        if word not in stopwords:
            filtered.append(word)
    return filtered


In [110]:
#JUNTANDO TODAS AS PALAVRAS DOS TWEETS RELEVANTES DO TREINAMENTO
train_relevante = train.loc[train["CLASSIFICAÇÃO"] == 1, :]
train_relevante_filtrada = ' '.join(train_relevante.TREINAMENTO) 

#JUNTANDO TODAS AS PALAVRAS DOS TWEETS IRRELEVANTES DO TREINAMENTO
train_irrelevante = train.loc[train["CLASSIFICAÇÃO"] == 0, :]
train_irrelevante_filtrada = ' '.join(train_irrelevante.TREINAMENTO) 


In [120]:
#LIMPANDO VARIÁVEIS CRIADAS, COLOCANDO EM MINÚSCULO E AS GUARDANDO
relevante = cleanup(train_relevante_filtrada.lower())
relevantes = RemoveStopWords(relevante)

irrelevante = cleanup(train_irrelevante_filtrada.lower())
irrelevantes = RemoveStopWords(relevantes)

#JUNTANDO OS CONJUNTOS
total = relevantes + irrelevantes


In [121]:
#TRANSFORMANDO EM PD.SERIES
series_train_global = pd.Series(total)
series_train_relevante = pd.Series(relevantes)
series_train_irrelevante = pd.Series(irrelevantes)

In [122]:
#FREQUÊNCIAS RELATIVAS
train_global_relativas = series_train_global.value_counts(True)
train_relevante_relativas = series_train_relevante.value_counts(True)
train_irrelevante_relativas = series_train_irrelevante.value_counts(True)

In [123]:
#FREQUÊNCIAS ABSOLUTAS
train_relevante_absolutas = series_train_relevante.value_counts()
train_irrelevante_absolutas = series_train_irrelevante.value_counts()
train_global_absolutas = series_train_global.value_counts()

**EVENTOS**

 * $R$: comentários relevantes
 * $R^c$: comentários irrelevantes
 * $C$: comentário recebido

**PROBABILIDADES**

 * $P(R)$ : probabilidade do comentário ser relevante ;
 * $P(R^c)$: probabilidade do comentário não ser relevante, ou seja, irrelevante ;
 * $P(C)$ : probabilidade de cada palavra ocorrer na língua portuguesa ;
 * $P(C|R)$ : probabilidade do comentário existir dado o conjunto de relevantes ;
 * $P(C|R^c)$: probabilidade do comentário existir dado o conjunto de irrelevantes ; 
 * $P(R|C)$: probabilidade do comentário ser relevante dado a frase ;
 * $P(R^c|C)$: probabilidade do comentário ser irrelevante dado a frase;


In [124]:
P_R = (train_relevante_absolutas.sum())/(train_global_absolutas.sum())
P_Rc =  (train_irrelevante_absolutas.sum())/(train_global_absolutas.sum())
P_R + P_Rc

1.0

In [128]:
prob_C_dado_R = train_relevante_relativas[total].prod()


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**